## Perkenalan
Nama : Fadhilah Amani Alam Aulia

Batch : 15

In [150]:
# Mengimport library yang dibutuhkan

import pandas as pd
import numpy as np
from scipy import stats

In [3]:
# Menghubungkan BigQuery dengan Google Colab

from google.colab import auth
from google.cloud import bigquery
auth.authenticate_user()
print('Authenticated')

project_id = "personal-fadhilah"
client = bigquery.Client(project=project_id)

Authenticated


In [ ]:
# Melakukan query dengan SQL

df = client.query('''
SELECT extract(year from created_at) as year, extract(month from created_at) as month, count(order_id) as sales
FROM `bigquery-public-data.thelook_ecommerce.orders`
WHERE status='Shipped' and created_at<"2022-07-01"
GROUP BY year, month
ORDER BY year, month ASC
''').to_dataframe()
df

,year,month,sales
0,2019,1,9
1,2019,2,38
2,2019,3,49
3,2019,4,72
4,2019,5,101
5,2019,6,98
6,2019,7,138
7,2019,8,146
8,2019,9,165
9,2019,10,195


---
## Poin Penjabaran
1. Berapa persen jumlah user yang retention dari Januari - Juni 2022 dari total keseluruhan user yang ada? (Asumsi definisi retention minimal melakukan transaksi 3 kali selama periode) Apa kesimpulan/insight yang bisa kamu berikan?

In [ ]:
# Menampilkan data user retention

df = client.query('''
SELECT *
FROM (
      SELECT users.first_name AS first_name,
             users.last_name AS last_name,
             orders.user_id AS user_id,
             COUNT(orders.order_id) AS frekuensi
      FROM `bigquery-public-data.thelook_ecommerce.users` users
      JOIN `bigquery-public-data.thelook_ecommerce.orders` orders
      ON users.id = orders.user_id
      WHERE orders.created_at BETWEEN "2022-01-01" AND "2022-06-30"
      GROUP BY users.first_name, users.last_name, orders.user_id
      ORDER BY frekuensi ASC
) sub
WHERE sub.frekuensi >= 3
''').to_dataframe()
df

,first_name,last_name,user_id,frekuensi
0,Karina,Owens,40066,3
1,Daniel,Edwards,19495,3
2,Joseph,Brown,52413,3
3,Amanda,Moore,71335,3
4,Erin,Parker,71458,3
...,...,...,...,...
1042,Michael,Miller,71592,4
1043,Justin,Nielsen,66196,4
1044,Brianna,Todd,87293,4
1045,Isaiah,Lopez,12616,4


In [ ]:
# Menghitung Persentase Jumlah User Retention

df = client.query('''
SELECT jml_retention, total_user, ROUND((jml_retention/total_user * 100),2) AS persentase
FROM(  
     ( 
      SELECT COUNT(*) as jml_retention
      FROM(
           SELECT users.first_name, 
                  users.last_name, 
                  orders.user_id, 
                  COUNT(orders.order_id) AS frekuensi
           FROM `bigquery-public-data.thelook_ecommerce.users` users
           JOIN `bigquery-public-data.thelook_ecommerce.orders` orders
           ON users.id = orders.user_id
           WHERE orders.created_at BETWEEN "2022-01-01" AND "2022-06-30"
           GROUP BY users.first_name, users.last_name, orders.user_id
           ORDER BY frekuensi ASC
          ) sub
        WHERE sub.frekuensi >= 3
    )
CROSS JOIN
    (
     SELECT COUNT(DISTINCT user_id) as total_user
     FROM `bigquery-public-data.thelook_ecommerce.orders`
     WHERE created_at BETWEEN "2022-01-01" AND "2022-06-30"
    )
)
''').to_dataframe()
df

,jml_retention,total_user,persentase
0,1047,30756,3.4


> Diasumsikan bahwa user retention adalah user yang telah melakukan transaksi lebih dari 3 kali. Selama periode Januari - Juni 2022, terdapat 1047 user retention dari 30756 total user, atau sebesar 3.4%.



2. Berikan informasi rata-rata, minimum, dan maksimum jumlah transaksi yang dilakukan oleh user loyal? (Tampilkan hanya dalam satu dataframe dan berdasarkan satu query select SQL) Informasi apa yang bisa kamu berikan?

In [ ]:
# Menghitung nilai rata-rata, minimum, dan maksimum jumlah transaksi oleh user loyal

df = client.query('''
SELECT ROUND(AVG(sub.frekuensi),2) as rata_rata,
       MIN(sub.frekuensi) as nilai_minimum,
       MAX(sub.frekuensi) as nilai_maksimum
FROM (
      SELECT users.first_name, 
             users.last_name, 
             orders.user_id, 
             COUNT(orders.order_id) AS frekuensi
      FROM `bigquery-public-data.thelook_ecommerce.users` users
      JOIN `bigquery-public-data.thelook_ecommerce.orders` orders
      ON users.id = orders.user_id
      WHERE orders.created_at BETWEEN "2022-01-01" AND "2022-06-30"
      GROUP BY users.first_name, users.last_name, orders.user_id
      ORDER BY frekuensi ASC
) sub
WHERE sub.frekuensi >= 3
''').to_dataframe()
df

,rata_rata,nilai_minimum,nilai_maksimum
0,3.18,3,4


> Rata-rata user loyal melakukan transaksi di The Look adalah sebanyak 3.18 kali, dengan frekuensi minimum transaksi sebanyak 3 kali dan frekuensi maksimum transaksi sebanyak 4 kali.



3. Berikan informasi rata-rata, minimum, dan maksimum uang yang dihabiskan oleh user loyal? (Tampilkan hanya dalam satu dataframe dan berdasarkan satu query select SQL) Informasi apa yang bisa kamu berikan?

In [ ]:
# Menghitung nilai rata-rata, minimum, dan maksimum uang yang dihabiskan oleh user loyal

df = client.query('''
SELECT ROUND(AVG(jumlah_uang),2) as rata_rata,
       ROUND(MIN(jumlah_uang),2) as nilai_minimum,
       ROUND(MAX(jumlah_uang),2) as nilai_maksimum
FROM (
      SELECT users.first_name, 
             users.last_name, 
             orders.user_id, 
             COUNT(orders.order_id) AS frekuensi,
             SUM(order_items.sale_price) as jumlah_uang
      FROM `bigquery-public-data.thelook_ecommerce.users` users
      JOIN `bigquery-public-data.thelook_ecommerce.orders` orders
      ON users.id = orders.user_id
      JOIN `bigquery-public-data.thelook_ecommerce.order_items` order_items
      ON orders.user_id = order_items.user_id 
      WHERE orders.created_at BETWEEN "2022-01-01" AND "2022-06-30"
      GROUP BY users.first_name, users.last_name, orders.user_id
      ORDER BY frekuensi ASC
) sub
WHERE sub.frekuensi >= 3
''').to_dataframe()
df

,rata_rata,nilai_minimum,nilai_maksimum
0,376.92,26.72,4272.78


> Rata-rata pengeluaran dari user loyal untuk melakukan transaksi di The Look adalah 376.92 USD dengan nilai minimum transaksi adalah 26.72 USD dan nilai maksimum transaksi adalah 4272.78 USD.

4. Berasal dari sumber trafik mana saja kebanyakan user loyal akses platform The Look? Informasi apa yang bisa kamu berikan?

In [ ]:
# Menampilkan data sumber trafik yang banyak digunakan oleh user loyal

df = client.query('''
SELECT traffic_source, COUNT(traffic_source) as jumlah
FROM (
      SELECT users.first_name AS first_name,
             users.last_name AS last_name,
             orders.user_id AS user_id,
             COUNT(orders.order_id) AS frekuensi,
             users.traffic_source AS traffic_source
      FROM `bigquery-public-data.thelook_ecommerce.users` users
      JOIN `bigquery-public-data.thelook_ecommerce.orders` orders
      ON users.id = orders.user_id
      WHERE orders.created_at BETWEEN "2022-01-01" AND "2022-06-30"
      GROUP BY users.first_name, users.last_name, orders.user_id, users.traffic_source
      ORDER BY frekuensi ASC
) sub
WHERE sub.frekuensi >= 3
GROUP BY traffic_source
ORDER BY jumlah DESC
''').to_dataframe()
df

,traffic_source,jumlah
0,Search,724
1,Organic,154
2,Facebook,66
3,Email,63
4,Display,40


> Sumber trafik (traffic source) yang paling banyak digunakan oleh user loyal adalah Search dengan jumlah user sebanyak 724 user, lalu diikuti dengan Organic, Facebook, Email, dan Display.

5. Apakah ada hubungan antara sumber trafik dengan loyal atau tidaknya seorang user? (Gunakan chi-square test untuk menguji hubungan itu) Hint: Output tabel query hanya ada dua kolom yaitu kolom 'trafic source' dan 'loyal' (kolom 'loyal' berisikan ya dan tidak saja)

In [129]:
# Membuat tabel data user, traffic source dan frekuensi transaksi

a = client.query('''
SELECT (users.traffic_source), sub.user_id, sub.frekuensi
FROM (
      SELECT orders.user_id AS user_id,
             COUNT(orders.order_id) AS frekuensi
      FROM `bigquery-public-data.thelook_ecommerce.users` users
      JOIN `bigquery-public-data.thelook_ecommerce.orders` orders
      ON users.id = orders.user_id
      WHERE orders.created_at BETWEEN "2022-01-01" AND "2022-06-30"
      GROUP BY orders.user_id
) sub
JOIN `bigquery-public-data.thelook_ecommerce.users` users
ON sub.user_id = users.id
''').to_dataframe()

# Mengubah tabel menjadi dataframe data

data = pd.DataFrame(a)
data

,traffic_source,user_id,frekuensi
0,Email,35122,1
1,Email,40345,1
2,Email,65104,1
3,Email,7342,1
4,Email,3739,1
...,...,...,...
30751,Facebook,47178,1
30752,Facebook,82325,1
30753,Facebook,94320,1
30754,Facebook,25785,1


In [142]:
# Membuat kolom 'loyal' dengan kondisi jika nilai 'frekuensi' lebih dari 3, maka 'Yes' dan jika kurang dari 3, maka 'No'

data['loyal'] = np.where(data['frekuensi'] >= 3, 'Yes', 'No')

# Menampilkan data hubungan antara traffic_source dengan loyal

data[['traffic_source','loyal']]

,traffic_source,loyal
0,Email,No
1,Email,No
2,Email,No
3,Email,No
4,Email,No
...,...,...
30751,Facebook,No
30752,Facebook,No
30753,Facebook,No
30754,Facebook,No


In [146]:
# Menampilkan jumlah data 'Yes' dan 'No'

data['loyal'].value_counts()

No     29709
Yes     1047
Name: loyal, dtype: int64

In [147]:
# Membuat tabel kontingensi untuk perhitungan chi-square

contingency_table=pd.crosstab(data["traffic_source"],data["loyal"])
contingency_table

loyal,No,Yes
traffic_source,,
Display,1168,40
Email,1474,63
Facebook,1731,66
Organic,4418,154
Search,20918,724


In [149]:
# Melakukan chi-square test

stat, p, dof, expected = stats.chi2_contingency(contingency_table)
print('p=%.3f' % (p))
if p > 0.05:
    print('Probably independent')
else:
    print('Probably dependent')

p=0.570
Probably independent


> Berdasarkan perhitungan di atas, nilai p = 0.570 dan p > 0.05, maka bisa dikatakan bahwa tidak terdapat hubungan yang signifikan antara traffic source dan tingkat loyal seorang user.

---
## Poin Analisis
1. Berikan kesimpulan dari laporan/informasi yang dibuat berdasarkan problem statement/poin penjabaran menggunakan bahasa awam! (boleh dalam beberapa paragraf)
> Berdasarkan problem statement/poin penjabaran, maka dapat disimpulkan bahwa:
  - Selama periode Januari - Juni 2022, terdapat 1047 user The Look yang melakukan retention dari total user sebanyak 30756 user, atau sebesar 3.4%. 
  - Rata-rata user loyal melakukan transaksi di The Look adalah sebanyak 3.18 kali, dengan frekuensi minimum transaksi sebanyak 3 kali dan frekuensi maksimum transaksi sebanyak 4 kali. 
  - Rata-rata pengeluaran dari user loyal untuk melakukan transaksi di The Look adalah 376.92 USD dengan nilai minimum transaksi adalah 26.72 USD dan nilai maksimum transaksi adalah 4272.78 USD. 
  - Sumber trafik (traffic source) yang paling banyak digunakan oleh user loyal adalah Search, Organic, Facebook, Email, dan yang terakhir adalah Display.
  - Hasil perhitungan Chi-Square terhadap data traffic source dan data loyal menghasilkan nilai p = 0.570 dan p > 0.05, maka bisa dikatakan bahwa tidak terdapat hubungan yang signifikan antara traffic source dan tingkat loyal seorang user.


---
## Poin Pertanyaan
1. Apakah problem statement yang kamu definisikan di awal dapat terukur ketercapaiannya? berikan pendapatmu!
> Ya, bisa. Karena kita sudah mengetahui data persentase user retention platform The Look serta rata-rata transaksi dan pengeluaran yang dilakukan oleh user loyal. Dengan mengetahui data-data tersebut, perusahaan dapat meningkatkan pelayanannya untuk meningkatkan persentase user retention. 
2. Apa yang dimaksud dengan customer/user retention?
> Customer/user retention adalah berbagai tindakan yang dilakukan perusahaan untuk mempertahankan customer agar tetap percaya dengan perusahaan, agar customer/user tetap menggunakan produk atau layanan mereka.
3. Jika platform The Look membuat fitur tambahan, apakah kamu akan merekomendasikan/memprioritaskan kepada user loyal? apa pertimbanganmu?
> Ya, saya akan merekomendasikan dan memprioritaskannya kepada user loyal. Karena menurut saya pertimbangan untuk membuat fitur tambahan tersebut didasari oleh permintaan atau saran dari user loyal. Selain itu, sebagai user yang sering mengakses platform, user loyal dapat memberikan feedback yang jujur mengenai layanan fitur tambahan tersebut.